<a href="https://colab.research.google.com/github/cleberjamaral/autoOrgDesignProject/blob/master/GoOrg4Prod_reallocation_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
import pandas as pd
import numpy as np
import altair as alt # for charts

In [56]:
bound_df = pd.read_csv("output/statistics/Feed_production_line_binding.csv","\t")
skills = ["move","lift","pnp"]
#skills = ["db_access","move","lift","pnp"]
#print(bound_df["pTree"])

In [57]:
def powerset(s):
    P = []
    x = len(s)
    for i in range(1 << x):
        P.append([s[j] for j in range(x) if (i & (1 << j))])
    return P

In [58]:
def get_organisations(array_str):
    organisations = []
    for i in range(0,len(array_str)):
        positions = []
        positions_str = array_str[i].split("G{")
        for j in range(0,len(positions_str)):
            position_str = positions_str[j].replace("[","").replace("]","").replace("}","").replace("^","").split(",")
            #print("p",j,position_str)
            position = []
            for k in range(0,len(position_str)):
                goal = position_str[k].strip()
                skill = goal.replace("MoveBox","move").replace("GetBox","lift").replace("PlaceBox","pnp")
                #print("g",k,goal)
                if len(goal)>0:
                    position.append(skill)
                #print(position)
            if len(position)>0:
                positions.append(position)
        #print(positions)
        organisations.append(positions)
    return organisations

In [59]:
bound_df.head()

,id,nPosit,%Idle+,%Effic,%Geral,%Speci,%Tall,%Flat,%Feasi,Levels,...,rWL,bDL,rDL,%WL+,%DL+,pTree,bgTree,skTree,agents,matche
0,1,1,46%,54%,100%,0%,0%,100%,0%,1,...,13.0,16.0,0.0,100%,0%,"[G{[GetBox, MoveBox, PlaceBox]}]","[PlaceBox, GetBox, MoveBox]","[[pnp, move, lift]]","[pp, bt, ie]",NaN
1,2,2,73%,27%,25%,75%,0%,100%,50%,1,...,13.0,16.0,8.0,100%,50%,"[G{[GetBox, MoveBox]}, G{[PlaceBox]}]","[PlaceBox, GetBox, MoveBox]","[[move, lift], [pnp]]","[pp, bt, ie]",p1=pp
2,3,2,73%,27%,25%,75%,0%,100%,50%,1,...,13.0,16.0,16.0,100%,100%,"[G{[GetBox, PlaceBox]}, G{[MoveBox]}]","[PlaceBox, GetBox, MoveBox]","[[move], [pnp, lift]]","[pp, bt, ie]",p1=bt
3,4,2,73%,27%,25%,75%,0%,100%,50%,1,...,13.0,16.0,8.0,100%,50%,"[G{[GetBox]}, G{[MoveBox, PlaceBox]}]","[PlaceBox, GetBox, MoveBox]","[[lift], [pnp, move]]","[pp, bt, ie]",p1=ie
4,5,2,73%,27%,25%,75%,50%,50%,50%,2,...,13.0,16.0,8.0,100%,50%,"[G{[GetBox, MoveBox]}, G{[PlaceBox]}^G{[GetBox...","[PlaceBox, GetBox, MoveBox]","[[move, lift], [pnp]]","[pp, bt, ie]",p1=pp


In [60]:
# Test if all positions generated by GoOrg4Prod are indeed in the
# set of all_agent_kinds
def test_all_orgs_match_agents_power_set():
    all_match = True
    orgs = get_organisations(bound_df["pTree"])

    all_agent_kinds = powerset(skills)
    np.savetxt("all_agent_kinds.csv",all_agent_kinds,delimiter = "\t",fmt = "% s")

    #for i in range(0,len(all_agent_kinds)):
    #    print(all_agent_kinds[i])
    for o in range(0,len(orgs)):
        #print(o,orgs[o])   
        for p in orgs[o]:
            contains = False
            for k in all_agent_kinds:
                if list(set(p)) in [list(set(k))]:
                    #print(":::",list(set(p)),"isin",all_agent_kinds)
                    contains = True

            if not contains:
                print(">>>",list(set(p)),":notin:",all_agent_kinds)
                all_match = False

    return all_match
    

In [61]:
print("orgs match agents powerset?",test_all_orgs_match_agents_power_set())

orgs match agents powerset? True


/usr/lib/python3/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [73]:
# For each possible set of agents (from the list that contains 65536 sets) this procedure
# checks if the set is applicable to every generated structure (the list contains 1646 structures)
# It also checks the overqualification cost of the matches
all_match_sets = []
applicables = []
count_match = []
count_zero_q_cost = []
match_set = []
all_agent_kinds = powerset(skills)
all_sets_of_agents = powerset(all_agent_kinds)

def check_all_applicable_sets_of_agents():
    orgs = get_organisations(bound_df["pTree"])

    for sa in range(0,len(all_sets_of_agents)):

        if (sa % 200) == 0:
            print(sa+1,"of",len(all_sets_of_agents),"items processed.")
        # Sort set to make sure to test first the sets that are cheaper in terms of qualification cost
        sorted_set = sorted(all_sets_of_agents[sa], key = len)
        
        print(sorted_set)

        # Check if the set has all necessary skills, which is a condition to be applicable
        # If there is any skill missing, this set would not be relevant, no organisation will be feasible
        all_sets_of_agents_flat = [val for sublist in sorted_set for val in sublist]
        if all(elem in all_sets_of_agents_flat for elem in skills):
            applicables.append(1)
        else:
            applicables.append(0)

        orgs = get_organisations(bound_df["pTree"])
        matches = 0
        zero_q_cost = 0
        for o in range(0,len(orgs)):
            all_match = True
            overqualification_cost = 0
            for p in orgs[o]:
                match = False
                for k in sorted_set:
                    if all(elem in set(k) for elem in set(p)):
                        match = True
                        overqualification_cost += len(set(k))-len(set(p))
                        break
                if not match:
                    all_match = False
                    break
            if all_match:
                matches += 1
                match_set.append([sa+1,o+1,overqualification_cost])
                if (overqualification_cost == 0):
                    zero_q_cost += 1
        count_match.append(matches)
        count_zero_q_cost.append(zero_q_cost)
        
check_all_applicable_sets_of_agents()

all_match_sets.append(match_set)
np.savetxt("all_sets_of_agents.csv",list(zip(all_sets_of_agents,applicables,count_match,all_match_sets,count_zero_q_cost)),delimiter = "\t",fmt = "% s")

1 of 256 items processed.
[]
[[]]
[['move']]
[[], ['move']]
[['lift']]
[[], ['lift']]
[['move'], ['lift']]
[[], ['move'], ['lift']]
[['move', 'lift']]
[[], ['move', 'lift']]
[['move'], ['move', 'lift']]
[[], ['move'], ['move', 'lift']]
[['lift'], ['move', 'lift']]
[[], ['lift'], ['move', 'lift']]
[['move'], ['lift'], ['move', 'lift']]
[[], ['move'], ['lift'], ['move', 'lift']]
[['pnp']]
[[], ['pnp']]
[['move'], ['pnp']]
[[], ['move'], ['pnp']]
[['lift'], ['pnp']]
[[], ['lift'], ['pnp']]
[['move'], ['lift'], ['pnp']]
[[], ['move'], ['lift'], ['pnp']]
[['pnp'], ['move', 'lift']]
[[], ['pnp'], ['move', 'lift']]
[['move'], ['pnp'], ['move', 'lift']]
[[], ['move'], ['pnp'], ['move', 'lift']]
[['lift'], ['pnp'], ['move', 'lift']]
[[], ['lift'], ['pnp'], ['move', 'lift']]
[['move'], ['lift'], ['pnp'], ['move', 'lift']]
[[], ['move'], ['lift'], ['pnp'], ['move', 'lift']]
[['move', 'pnp']]
[[], ['move', 'pnp']]
[['move'], ['move', 'pnp']]
[[], ['move'], ['move', 'pnp']]
[['lift'], ['move', 'pnp

/usr/lib/python3/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [67]:
match_set

[[23, 11, 0],
 [23, 12, 0],
 [23, 13, 0],
 [23, 14, 0],
 [23, 15, 0],
 [23, 16, 0],
 [23, 17, 0],
 [23, 18, 0],
 [23, 19, 0],
 [23, 20, 0],
 [23, 21, 0],
 [23, 22, 0],
 [23, 23, 0],
 [23, 24, 0],
 [23, 25, 0],
 [23, 26, 0],
 [24, 11, 0],
 [24, 12, 0],
 [24, 13, 0],
 [24, 14, 0],
 [24, 15, 0],
 [24, 16, 0],
 [24, 17, 0],
 [24, 18, 0],
 [24, 19, 0],
 [24, 20, 0],
 [24, 21, 0],
 [24, 22, 0],
 [24, 23, 0],
 [24, 24, 0],
 [24, 25, 0],
 [24, 26, 0],
 [25, 2, 0],
 [25, 5, 0],
 [25, 9, 0],
 [25, 11, 2],
 [25, 12, 2],
 [25, 13, 2],
 [25, 14, 3],
 [25, 15, 3],
 [25, 16, 3],
 [25, 17, 3],
 [25, 18, 2],
 [25, 19, 4],
 [25, 20, 4],
 [25, 21, 4],
 [25, 22, 5],
 [25, 23, 3],
 [25, 24, 3],
 [25, 25, 5],
 [25, 26, 4],
 [26, 2, 0],
 [26, 5, 0],
 [26, 9, 0],
 [26, 11, 2],
 [26, 12, 2],
 [26, 13, 2],
 [26, 14, 3],
 [26, 15, 3],
 [26, 16, 3],
 [26, 17, 3],
 [26, 18, 2],
 [26, 19, 4],
 [26, 20, 4],
 [26, 21, 4],
 [26, 22, 5],
 [26, 23, 3],
 [26, 24, 3],
 [26, 25, 5],
 [26, 26, 4],
 [27, 2, 0],
 [27, 5, 0],


In [112]:
df = pd.DataFrame(match_set, columns=["id_a", "id_b", "cost"])

In [113]:
# Plotting configurations
CHART_WIDTH = 800
CHART_HEIGHT = 200

In [117]:
dfplot = df.groupby(['id_a']).size().reset_index(name='counts')
       
dfplot

,id_a,counts
0,23,16
1,24,16
2,25,19
3,26,19
4,27,19
...,...,...
213,252,26
214,253,26
215,254,26
216,255,26


In [118]:
for i in range(0,max(df['id_a'])):
    if (df[(df['id_a']==i)].value_counts().empty):
        dfplot = dfplot.append({'id_a': i, "counts":0}, ignore_index=True)
dfplot

,id_a,counts
0,23,16
1,24,16
2,25,19
3,26,19
4,27,19
...,...,...
252,70,0
253,81,0
254,82,0
255,85,0


In [119]:
alt.data_transformers.disable_max_rows()
chart = alt.Chart(dfplot).mark_bar().encode(
    x=alt.X('id_a:O', axis=alt.Axis(title="")),
    y=alt.Y('counts:Q', axis=alt.Axis(title=""))
).properties(
    title=["Satisfatibility"]
)

text = chart.mark_text(baseline='middle').encode(
    #text='Cost:Q',
    color=alt.condition(
        alt.datum.Cost < 70,
        alt.value('black'),
        alt.value('white')
    ),
    #size=alt.value(6),
    opacity=alt.value(0.85)
)

# Draw the chart
plot = chart.properties(width=CHART_WIDTH, height=CHART_HEIGHT) + text
plot

alt.LayerChart(...)

In [74]:
dfplot = df.groupby(['id_b']).size().reset_index(name='counts')

alt.data_transformers.disable_max_rows()
chart = alt.Chart(dfplot).mark_bar().encode(
    x=alt.X('id_b:O', axis=alt.Axis(title="")),
    y=alt.Y('counts:Q', axis=alt.Axis(title=""))
).properties(
    title=["Satisfatibility"]
)

text = chart.mark_text(baseline='middle').encode(
    #text='Cost:Q',
    color=alt.condition(
        alt.datum.Cost < 70,
        alt.value('black'),
        alt.value('white')
    ),
    #size=alt.value(6),
    opacity=alt.value(0.85)
)

# Draw the chart
plot = chart.properties(width=CHART_WIDTH, height=CHART_HEIGHT) + text
plot

alt.LayerChart(...)

In [66]:
dfplot = pd.DataFrame({'x': df["id_a"].ravel(),
                   'y': df["id_b"].ravel(),
                   'Cost': df["cost"].ravel()})
alt.data_transformers.disable_max_rows()
chart = alt.Chart(dfplot).mark_rect().encode(
    x=alt.X('x:O', axis=alt.Axis(title="")),
    y=alt.Y('y:O', axis=alt.Axis(title=""), sort="descending"),
    color='Cost:Q'
).properties(
    title=["Overqualification cost"]
)

text = chart.mark_text(baseline='middle').encode(
    #text='Cost:Q',
    color=alt.condition(
        alt.datum.Cost < 70,
        alt.value('black'),
        alt.value('white')
    ),
    #size=alt.value(6),
    opacity=alt.value(0.85)
)

# Draw the chart
plot = chart.properties(width=CHART_WIDTH, height=CHART_HEIGHT) + text
plot

alt.LayerChart(...)

In [12]:
# For each organisation, check which kinds of agent it has
all_agent_kinds = powerset(skills)
all_kinds = []
orgs = get_organisations(bound_df["pTree"])

def check_all_kinds_of_agents_in_orgs():
    for o in range(0,len(orgs)):
        kinds = []
        for k in range(0,len(all_agent_kinds)):
            if all_agent_kinds[k] in orgs[o]:
                kinds.append(k)
                #print(all_agent_kinds[k],">>",orgs[o],"::",kinds)
        all_kinds.append(kinds)
        
check_all_kinds_of_agents_in_orgs()

In [13]:
# Flatness similarity
dfsimilarity = pd.DataFrame(columns=["id_a", "id_b", "similarity"])
dfsimilarity.columns = dfsimilarity.columns.map(str)

# Process correlations for table with 3 columns
for i in range(len(bound_df["id"])):
    for j in range(len(bound_df["id"])):
        dfsimilarity.loc[(i*len(bound_df["id"]))+j] = (
            bound_df["id"][i], 
            bound_df["id"][j],
            round(100*(max(all_kinds)[0]-len(list(set(all_kinds[i]) - set(all_kinds[j])) + list(set(all_kinds[j]) - set(all_kinds[i]))))/max(all_kinds)[0])
         )

In [14]:
# Plotting configurations
CHART_WIDTH = 800
CHART_HEIGHT = 600

In [17]:
dfplot = pd.DataFrame({'x': dfsimilarity["id_a"].ravel(),
                   'y': dfsimilarity["id_b"].ravel(),
                   'Difference': dfsimilarity["similarity"].ravel()})
#alt.data_transformers.disable_max_rows()
chart = alt.Chart(dfplot).mark_rect().encode(
    x=alt.X('x:O', axis=alt.Axis(title="")),
    y=alt.Y('y:O', axis=alt.Axis(title=""), sort="descending"),
    color='Difference:Q'
).properties(
    title=["Kind of Agents Similarity Comparison"]
)

text = chart.mark_text(baseline='middle').encode(
    text='Difference:Q',
    color=alt.condition(
        alt.datum.Difference < 70,
        alt.value('black'),
        alt.value('white')
    ),
    size=alt.value(10),
    opacity=alt.value(0.85)
)

# Draw the chart
plot = chart.properties(width=CHART_WIDTH, height=CHART_HEIGHT) + text
plot

alt.LayerChart(...)